# FHIR Digitally Signing FHIR Bundle or QuestionnaireResponse Object

This is a Jupyter Notebook using Python 3.7 and openSSl to create JSON Web Signature (JWS)(see RFC 7515) and attach it to a FHIR Bundle or QuestionnaireResponse resource.

- If the resource is a Bundle use Bundle.signature
- If the resource is a QuestionnaireResponse use its [Signature extension](http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature
)

See enveloped signatures: http://build.fhir.org/signatures.html

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

## Sender/Signer Steps

### 1. Generate RSA256 public and private keys for signing the bundle

**DO THIS STEP ONLY ONCE-**

### 2. Create a sef-signed certificate for authenticating the signer

create the public and private keys and cert using openssl on the command line.

1. pre-configure the self-signed cert with a configuration file

~~~
[req]
default_bit = 4096
distinguished_name = req_distinguished_name
prompt = no
x509_extensions = v3_ca

[req_distinguished_name]
countryName             = US
stateOrProvinceName     = California
localityName            = Sausalito
organizationName        = HealtheData1
commonName              = Eric Haas, DVM
emailAddress            = ehaas@healthedata1.org

[v3_ca]
basicConstraints = CA:TRUE
keyUsage=nonRepudiation, digitalSignature, keyEncipherment
subjectAltName = @alt_names

[alt_names]
DNS.1 = www.healthedatainc.com
~~~

2\. generate the public and private keys and cert

~~~
!openssl genrsa -out private-key.pem 3072
!openssl rsa -in private-key.pem -pubout -out public-key.pem
!openssl req -new -x509 -key private-key.pem -outform DER -out cert.der -days 360 -config cert.config
~~~

##### For the purpose of this example display the keys (normally would never share the private key)

##### Show the Certificate in DER Format

##### Show the Certicate in PEM format

### 3. Create JWS to Attach to Bundle or QR (START HERE IF USING THE SAME CERT)

#### 3.01  Auto Timestamp Function

In [ ]:
from datetime import datetime
import pytz

auto_timestamp = False
timezone= 'US/Pacific'

def get_timestamp(timezone):  # Get current time in timezone
  tz = pytz.timezone('US/Pacific')
  current_time = datetime.now(tz)
  iso_timestamp = current_time.isoformat()  # Format as ISO 8601 compliant string
  return(iso_timestamp)
# print(get_timestamp(timezone))

#### 3.1. Prepare Header

 note the base64 DER is Cert PEM file wihout the footer and header and line returns

In [ ]:


with open('cert.pem') as f:
    der = (f.read())  # base64 DER is PEM wihout the footer and header and line returns
der = der.replace('-----BEGIN CERTIFICATE-----','')
der = der.replace('-----END CERTIFICATE-----','')
der = der.replace('\n','')
header = {"alg": "RS256","kty": "RS", "sigT": get_timestamp(timezone) if auto_timestamp else '2020-10-23T04:54:56.048+00:00' ,"x5c": [der]}
header

### 3.2 Fetch Payload from FHIR Server or Local File

In [ ]:
from requests import get
from json import dumps, loads
from pathlib import Path
local_file = True


def fetch_payload():


  # bundle_id = "f50e72a6-290e-44f8-9b6f-adf19713d0d4/_history/183"  # insert bundle id here
  bundle_id = 'Bundle-cdex-searchbundle-digital-sig-example.json'

  # url = "https://argopatientlist.aidbox.app/fhir/Bundle"
  # url = 'http://test.fhir.org/r4/Bundle'
  url = 'https://hl7.org/fhir/us/davinci-cdex'

  username = "basic"
  password = "secret"
  headers = {"Accept": "application/fhir+json" , "Content-Type": "application/fhir+json"}

  r = get(f'{url}/{bundle_id}', auth=(username, password), headers = headers)
  my_obj = r.json()
  # print("="*80)
  # print("STATUS: ",r.status_code)

  # print("="*80)
  # print("HEADERS:\n")
  # for k,v in r.headers.items():
  #     print(f'{k} = {v}')
  # print("="*80)
  # print("BODY:\n")
  # print(dumps(my_obj,indent=2))
  return(my_obj)


if local_file: 
  # in_file = 'in_files/object_to_sign.json'
  in_file = '/Users/ehaas/Documents/FHIR/davinci-ecdx/output/Bundle-cdex-searchbundle-digital-sig-example.json'
  path = Path() / in_file
  my_obj =loads(path.read_text())
else:
  my_obj = fetch_payload()
print(dumps(my_obj,indent=2))

#### 3.2.1 Prepare Payload

The payload is the base64_url form of the canonicalized version of the resource before attaching the signature
 

#####  Canonicalize the resource using IETF JSON Canonicalization Scheme (JCS) before adding the signature element

- Remove the id, meta, and signature elements if present before canonicalization
- Note keep the the generated narrative if present

In [ ]:
from jcs import canonicalize #package for a JCS (RFC 8785) compliant canonicalizer.

def  pop_element(resource, element):
  try:
    my_element = resource.pop(element) # remove element
    return my_element 
  except KeyError:
    pass


if my_obj['resourceType'] in ['Bundle', 'QuestionnaireResponse']:
  my_obj_id = pop_element(my_obj, 'id')
  my_obj_meta = pop_element(my_obj, 'meta')
  my_obj_old_sig = pop_element(my_obj, 'signature')
else:
  print('Not a Bundle or QuestionnaireResponse')


if my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    for i, extension in enumerate(my_obj['extension']):
       if extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature':
          my_obj_signature_ext = my_obj['extension'].pop(i) # remove element
    if i == 0:
      my_obj.pop('extension') # remove extension if empty
  except KeyError:
    print('No signature extension found')

payload = canonicalize(my_obj)
payload, len(payload)

##### Then base64_url the payload entry

note this step is combined with 3.3 below using the jws.sign method.

#### 3.3 Create Signature using private key and the RS256 algorithm to get the JWS compact serialization format

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing.

In [ ]:
from jose import jws  #python JWS package
with open('private-key.pem') as f:
    private_key = (f.read())

signature = jws.sign(payload,private_key,algorithm='RS256',headers=header)

labels = ['header', 'payload', 'signature']
for i,j in enumerate(signature.split('.')):
    print(f'{labels[i]}:\n{j}\n')

#### 3.4. Option to Create "detached-content" payload by removing the payload from the JWS

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing then as compact serialization format

In [ ]:
detached = False
if detached:
  split_sig = signature.split('.')
  split_sig[1] = ''
  signature = '.'.join(split_sig)
  for i,j in enumerate(signature.split('.')):
      print(f'{labels[i]}:\n{j}\n')
print(f'\nSignature in compact serialization format:\n{"="*80}\n{signature}')

### 4. base64 the JWS and add the Signature element to the Bundle or QR

this is what would be contained and/or referenced by TASK over-the-wire

In [ ]:
from base64 import b64encode
from json import loads,dumps

b64_jws = b64encode(signature.encode()).decode()
sig_element = {
            "type": [  # Signature.type = Verification Signature
              {
                "system": "urn:iso-astm:E1762-95:2013",
                "code": "1.2.840.10065.1.12.1.5",
                "display": "Verification Signature"
              }
            ],
            "when": get_timestamp(timezone) if auto_timestamp else '2020-10-23T04:54:56.048+00:00', #system timestamp when signature created
            # "who" { #Reference to the Practitioner who signed and attested to the Bundle
            #   "reference": "Practitioner/123"  
            #   "display": "Practitioner/123" 
            # },
            # "onBehalfOf": { #Reference to the Organization
            #   "reference": "Organization/123"
            #   "display": "Organization/123"
            # },
            "who": { #Reference to the Practitioner who signed and attested to the Bundle using NPI
                    "identifier": {
                        "system": "http://hl7.org/fhir/sid/us-npi",
                        "type" : {
                            "coding" : [{
                                "system" : "http://terminology.hl7.org/CodeSystem/v2-0203",
                                "code" : "NPI"
                              }]
                          },
                        "value": "9941339108"
                    },
                    "display": "Dr Signer"
                },
            "onBehalfOf": {
                    "identifier": {
                        "system": "http://hl7.org/fhir/sid/us-npi",
                        "value": "1184932014"
                    }
                },
            "targetFormat" : "application/fhir+json;canonicalization=http://hl7.org/fhir/canonicalization/json#document", # The technical format of the signed resources see 
            #https:// hl7.org/fhir/json.html#canonical. The MIME types can include optional parameters in the format type/subtype;
            # parameter=value, as defined in RFC 2045 and RFC 6838. For example, text/plain;charset=utf-8 specifies a character encoding.
            "sigFormat" : "application/jose", # The technical format of the signature
            "data": b64_jws,
             }

if my_obj_id :
  my_obj['signature'] =  my_obj_id # update id
  my_obj['signature'] = my_obj_meta # update meta
if sig_element:
  my_obj['signature'] = sig_element # update signature

if my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    my_obj['extension'].append(dict(
        url='http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature',
        valueSignature=sig_element
        )) # update signature  
  except KeyError:
    my_obj['extension'] = [dict(
        url='http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature',
        valueSignature=sig_element
        )]

print(dumps(my_obj, indent=2, sort_keys=False))

### Using FHIR RESTful create POST to FHIR Server 
***Deactivated until until get new AIDBOX***
using AIDBox for now at `https://argopatientlist.aidbox.app/fhir/`
<!-- using `http://test.fhir.org/r4/` -->


### Alternatively Write to Local File as JSON

In [ ]:
out_path = Path(r'out_files/signed_object.json')
print(f'Writing signed object to {out_path}')
out_path.write_text(dumps(my_obj,indent=2,sort_keys=False))